In [1]:
import pandas as pd
import matplotlib.pyplot as plt

cpg_sample_df = pd.read_csv('cpg_sample_df.csv')
meth_df = pd.read_csv('../../output_data/DNAMethylation.csv')
cols = list(cpg_sample_df['Sample'].values)

meth_df = meth_df.replace(0, 0.001)
meth_df = meth_df.replace(1.0, 0.999)


NameError: name 'np' is not defined

In [2]:
import numpy as np
mean_meth = np.nanmean(meth_df[cols].values, axis=1)
meth_df = meth_df[mean_meth > 0.05]
mean_meth = np.nanmean(meth_df[cols].values, axis=1)


/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_69740/78074593.py:2: RuntimeWarning: Mean of empty slice
  mean_meth = np.nanmean(meth_df[cols].values, axis=1)


NameError: name 'mean_cor' is not defined

In [ ]:
meth_df = meth_df[mean_meth < 0.95]
corr = meth_df[cols].corr()
# Print out the minimum correlation:
mean_cor = np.nanmean(corr, axis=1)
corr['mean_corr'] = mean_cor
corr.sort_values(by=['mean_corr'])
# Plot out the mean correlation values so we can choose a good filter.
plt.hist(mean_cor, bins=20)
plt.title(f'min corr: {min(np.min(corr))}')

# Filter the shitty patients
corr_sorted = corr.sort_values(by=['mean_corr'])
corr_sorted = corr_sorted[corr_sorted['mean_corr'] < 0.5]
u.dp([len(corr_sorted), 'patients with avg. correlations less than 50%. Filtering out these samples, and printing cases.'])

cols_to_omit = [c for c in corr_sorted.index]
print('\n'.join(cols_to_omit))
cols_to_keep = [c for c in meth_df.columns if c not in cols_to_omit]
cpg_filtered = meth_df[cols_to_keep]

# Fill NAs with 0's
cpg_filtered = cpg_filtered.fillna(0)
# Filter sample df to only include those samples
sample_df = cpg_sample_df[cpg_sample_df['Sample'].isin(cols_to_keep)]

# Now we want to check each cancer has a separation between tumour and normal and all cancers also have a decent separation

In [ ]:
# Drop CpGs with > 50% missing values
df = meth_df[meth_df.isnull().sum(axis=1) < len(meth_df.values[0])/2]
cols = list(sample_df['Sample'].values)
vals = df[cols].values.T
# If there is a missing CpG replace it with the mean value for that CpG

pca = PCA(n_components=2)
pca_values = pca.fit_transform(vals)
var_ratio = pca.fit(vals).explained_variance_ratio_
plt.rcParams['figure.figsize'] = [4, 4]
vis_df = pd.DataFrame()
vis_df['PC_1'] = pca_values[:, 0]
vis_df['PC_2'] = pca_values[:, 1]
vis_df['Stage'] = sample_df['Stage'].values
vis_df['Disease'] = sample_df['Disease'].values
vis_df['CondID'] = sample_df['CondID'].values

sns.scatter(vis_df, 'PC_1', 'PC_2', hue=Stage, s=100, alpha=0.5)
plt.show()